In [7]:
import cv2
import numpy as np
import dlib
import matplotlib.pyplot as plt
import os
import tensorflow as tf 

In [8]:
def detect_face_landmarks(frame, face_detector, landmark_predictor):
    
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray_frame)
    
    landmarks_list = []
    for face in faces:
        landmarks = landmark_predictor(gray_frame, face)
        landmarks_points = [(landmarks.part(n).x, landmarks.part(n).y) for n in range(68)]
        landmarks_list.append(landmarks_points)
    
    return landmarks_list

In [9]:
def extract_eyes_from_landmarks(frame, landmarks):
    
    if len(landmarks) != 68:
        raise ValueError("Facial landmarks should contain 68 points.")

    # Define the indices for the left and right eyes based on facial landmarks.
    left_eye_indices = [i for i in range(36, 42)]
    right_eye_indices = [i for i in range(42, 48)]

    # Extract left and right eye regions from the frame.
    left_eye_coords = np.array([landmarks[i] for i in left_eye_indices], dtype=np.int32)
    right_eye_coords = np.array([landmarks[i] for i in right_eye_indices], dtype=np.int32)

    # Create a mask for each eye region.
    mask_left = np.zeros_like(frame)
    mask_right = np.zeros_like(frame)
    cv2.fillPoly(mask_left, [left_eye_coords], (255, 255, 255))
    cv2.fillPoly(mask_right, [right_eye_coords], (255, 255, 255))

    # Apply the mask to the frame to get the eye regions.
    left_eye_region = cv2.bitwise_and(frame, mask_left)
    right_eye_region = cv2.bitwise_and(frame, mask_right)

    # Convert the eye regions to grayscale for further processing if needed.
    left_eye_gray = cv2.cvtColor(left_eye_region, cv2.COLOR_BGR2GRAY)
    right_eye_gray = cv2.cvtColor(right_eye_region, cv2.COLOR_BGR2GRAY)

    return [left_eye_gray, right_eye_gray]

In [10]:
def preprocess_single_image(image_path, img_size):
    # Read the image in grayscale
    img_array = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

    # Resize the image while maintaining the aspect ratio
    desired_size = (img_size, img_size)
    height, width = img_array.shape
    aspect_ratio = width / height

    if aspect_ratio >= 1:
        new_width = desired_size[0]
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = desired_size[1]
        new_width = int(new_height * aspect_ratio)

    resized_image = cv2.resize(img_array, (new_width, new_height))

    # Pad the resized image to make it square (img_size x img_size)
    pad_width = (desired_size[1] - new_height) // 2
    pad_height = (desired_size[0] - new_width) // 2
    padded_image = np.pad(resized_image, ((pad_width, pad_width), (pad_height, pad_height)), mode='constant', constant_values=0)

    # Convert the grayscale image to RGB
    rgb_image = cv2.cvtColor(padded_image, cv2.COLOR_GRAY2RGB)

    return rgb_image
